In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
from regions import PixCoord

import pylab as pl

from scipy.optimize import curve_fit
from scipy.integrate import quad

from orb.fit import fit_lines_in_spectrum
from orb.utils.spectrum import corr2theta, amp_ratio_from_flux_ratio
from orb.core import Lines
import gvar
import orb

import orb.utils.io as io
import orb.utils.graph as graph

import pyregion

from scipy.integrate import simps

### Import Data

In [ ]:
galaxynum = 5
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  #There is no SITELLE data for NGC 4254, NGC 2835 has the best data 
galaxy = galdic[galaxynum]
print(galaxy)

galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651}
galvel = galveldic[galaxy]

infile = open("/home/habjan/jupfiles/data/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
musedata = hdul[hdul['gal_name'] == f'{galaxy}']

hdul = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra.fits')

lam = hdul[0].data
specextr = hdul[1].data

infile = fits.open(f'/home/habjan/jupfiles/data/Nebulae_Catalogue_DR2_native_with_OII.fits')
hdul = Table.read(infile)
fabian = hdul[hdul['gal_name'] == f'{galaxy}']

infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

# WCS info for SITELLE
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_cube.fits")
header = hdul[0].header
wcs = WCS(header,naxis=2)

# MUSE WCS info 
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_IMAGE_FOV_Johnson_B_WCS_Pall_mad.fits")
muse_data = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header))    
muse_data.data[muse_data.data==0] = np.nan

# Halpha masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_MAPS.fits")
Halpha = NDData(data=hdul['HA6562_FLUX'].data, mask=np.isnan(hdul['HA6562_FLUX'].data), meta=hdul['HA6562_FLUX'].header, wcs=WCS(hdul['HA6562_FLUX'].header))
Halpha.data[muse_data.data==0] = np.nan

# Import the HII region spatial masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_nebulae_mask_V2.fits")
nebulae_mask = NDData(data = hdul[0].data.astype(float), mask=Halpha.mask, meta=hdul[0].header, wcs=WCS(hdul[0].header)) 
nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# SITELLE deepframe 
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_deepframe.fits")

deepframe = NDData(data=hdul[0].data, meta=hdul[0].header, wcs=wcs)

### Find the Object in the Nebular catalog with the smallest deprojected radius and obtain pixel coordinates in the SITELLE image

In [ ]:
cenreg = np.where(np.min(musedata['deproj_dist']) == musedata['deproj_dist'])[0][0]
testcoord = WCS(nebulae_mask.meta).pixel_to_world(np.where(nebulae_mask.data == cenreg)[1], np.where(nebulae_mask.data == cenreg)[0])
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

### Plot the deep frame image of the galaxy

In [ ]:
deep = cube.get_deep_frame()
deep.imshow(perc=95, wcs=wcs)#, cmap='Greys')

pl.scatter(sitpix[0], sitpix[1], color='red')

pl.title(f'{galaxy} SITELLE deepframe')
pl.xlabel('R.A.')
pl.ylabel('Dec.')

### Determine inner and outer radii as well as the center of the annulus

In [ ]:
nebpix = (np.where(~np.isnan(nebulae_mask.data))[0], np.where(~np.isnan(nebulae_mask.data))[1])
testcoord = WCS(nebulae_mask.meta).pixel_to_world(nebpix[1], nebpix[0])
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

r = np.max([np.max(sitpix[0]) - np.min(sitpix[0]), np.max(sitpix[1]) - np.min(sitpix[1])]) / 2 
rmin = r * 3
if galaxy == 'NGC0628':
    rmin = r * 2.5
rmax = np.sqrt(r**2 + rmin**2)

cenreg = np.where(np.min(musedata['deproj_dist']) == musedata['deproj_dist'])[0][0]
nebpix = (np.where(nebulae_mask.data == cenreg)[0], np.where(nebulae_mask.data == cenreg)[1])
testcoord = WCS(nebulae_mask.meta).pixel_to_world(nebpix[1], nebpix[0])
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)
x = np.median(sitpix[0])
y = np.median(sitpix[1])

rmin, rmax, x, y

### code to find all of the regions in this annulus

In [ ]:
imin = x - rmax
imax = x + rmax + 1
jmin = y - rmax
jmax = y + rmax + 1
xlist = []
ylist = []
buffval = 50

for i in np.arange(imin, imax):
    if 0 + buffval <= i <= 2048 - buffval:
        for j in np.arange(jmin, jmax):
            if 0 + buffval <= j <= 2064 - buffval:
                ij = np.array([i,j])
                dist = np.linalg.norm(ij - np.array((x,y)))
                i, j = int(i), int(j)
                distnum = 3
                if dist > rmin and dist <= rmax:# and np.all(np.isnan(nebulae_mask.data[i-distnum:i+distnum,j-distnum:j+distnum])):
                    xlist.append(i)
                    ylist.append(j)

inpix = (xlist, ylist)

### Plot the spaxels in the annulus for visualization

In [ ]:
deep = cube.get_deep_frame()
deep.imshow(perc=95, wcs=wcs)#, cmap='Greys')

pl.scatter(inpix[0], inpix[1], color='red', marker='.', s=5, alpha = 0.01)

pl.title(f'{galaxy} SITELLE deep frame')
pl.xlabel('R.A.')
pl.ylabel('Dec.')

### Extract the average sky background per pixel within the annulus

In [ ]:
batchnum = 1000
annuluslist = [(inpix[0][i:i+batchnum], inpix[1][i:i+batchnum]) for i in range(0, len(inpix[0]), batchnum)]
speclist = []

for i in range(len(annuluslist)):
    spec = cube.extract_integrated_spectrum(annuluslist[i], mean_flux=True)
    speclist.append(np.real(spec[1]))

waves = np.real(spec[0])
specs = np.mean(speclist, axis=0)

### Here are some of the brightest regions in the galaxy. Find a spaxel in one of the regions to visualize

In [ ]:
brightreg = np.where(musedata['OII7319_FLUX'] > 30*musedata['OII7319_FLUX_ERR'])[0]
brightreg

### Use the background spectrum obtained from the SITELLE cube to subtract the background from an HII region spectrum from one of the above regions

In [ ]:
brightnum = 0
region = brightreg[brightnum]

nebpix = np.where(nebulae_mask.data == region)
testcoord = WCS(nebulae_mask.meta).pixel_to_world(nebpix[1], nebpix[0])
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

regscale = len(nebpix[0])    ## Number of pixels in HII region is used to rescale the background spectrum so that the appropriate amount of background is subtracted

specback = cube.extract_integrated_spectrum(sitpix, median=True)
specback = np.real(specback[1])
specback = specback - (regscale * specs)    ## Background is subtracted 

ave = specback / (specextr[region] * 10**-20)
ave = ave[np.where(~np.isnan(ave))]
np.mean(ave)

### Plot the sky background, an existing spectrum from an HII region, and now the background subtracted region

In [ ]:
low, up = np.where(cube.params.filter_range[0] < waves)[0][0], np.where(cube.params.filter_range[1] < waves)[0][0]
red3729 = 1/((3729*(galvel)/(299792) + 3729) * 10**-8)

fig = plt.figure()
fig.set_size_inches(10, 4)

plt.plot(waves, (specextr[region] * 10**-20), color='green', label = f'Region {region} Spectrum')
plt.plot(waves, specback, color='k',label=f'Sky Background subtracted Region {region}')
plt.plot(waves, specs * regscale, color='red', linestyle='--', label=f'Sky Background Flux (rescaled {regscale}x)')

#plt.plot(waves, (specextr[brightreg[region]] * 10**-20), color='green', label=f'Sky Subtracted Region {brightreg[region]}')
plt.axhline(0, color='k', label='0 line')
plt.axvline(red3729, color='blue', linestyle='--', label='Redshifted 3729 Angstrom')

plt.xlim(np.min(waves[low:up]), np.max(waves[low:up]))
plt.xlim(waves[low], waves[up])
#plt.ylim(3.5*10**-19, 7*10**-19)

plt.legend(bbox_to_anchor=(1.53, 0.5), fontsize="9",loc='center right');
#plt.yscale('log')
plt.title(f'{galaxy} sky background')
plt.xlabel(r'Wavenumber [cm$^{-1}$]')
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]')

### Create a function to extract a sky line background spectrum

In [ ]:
def skyback(incube, nmask, phdata, ingal):

    nebpix = (np.where(~np.isnan(nmask.data))[0], np.where(~np.isnan(nmask.data))[1])
    testcoord = WCS(nmask.meta).pixel_to_world(nebpix[1], nebpix[0])
    sitpix = np.transpose(incube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

    r = np.max([np.max(sitpix[0]) - np.min(sitpix[0]), np.max(sitpix[1]) - np.min(sitpix[1])]) / 2 
    rmin = r * 3
    if ingal == 'NGC0628':
        rmin = r * 2.5
    rmax = np.sqrt(r**2 + rmin**2)

    cenreg = np.where(np.min(phdata['deproj_dist']) == phdata['deproj_dist'])[0][0]
    nebpix = (np.where(nmask.data == cenreg)[0], np.where(nmask.data == cenreg)[1])
    testcoord = WCS(nmask.meta).pixel_to_world(nebpix[1], nebpix[0])
    sitpix = np.transpose(incube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)
    x = np.median(sitpix[0])
    y = np.median(sitpix[1])

    imin = x - rmax
    imax = x + rmax + 1
    jmin = y - rmax
    jmax = y + rmax + 1
    xlist = []
    ylist = []
    buffval = 50

    for i in np.arange(imin, imax):
        if 0 + buffval <= i <= 2048 - buffval:
            for j in np.arange(jmin, jmax):
                if 0 + buffval <= j <= 2064 - buffval:
                    ij = np.array([i,j])
                    dist = np.linalg.norm(ij - np.array((x,y)))
                    i, j = int(i), int(j)
                    distnum = 3
                    if dist > rmin and dist <= rmax:# and np.all(np.isnan(nmask.data[i-distnum:i+distnum,j-distnum:j+distnum])):
                        xlist.append(i)
                        ylist.append(j)

    inpix = (xlist, ylist)

    batchnum = 1000 
    annuluslist = [(inpix[0][i:i+batchnum], inpix[1][i:i+batchnum]) for i in range(0, len(inpix[0]), batchnum)]
    speclist = []

    for i in range(len(annuluslist)):
        spec = incube.extract_integrated_spectrum(annuluslist[i], mean_flux=True)

        speclist.append(np.real(spec[1]))

    return np.median(speclist, axis=0)

### Extract the [OII] fluxes

In [ ]:
inspec = (specextr[brightreg[region]] * 10**-20) - specs

OII3726 = Lines().get_line_cm1('[OII]3726')
OII3729 = Lines().get_line_cm1('[OII]3729')

invel = musedata[region]['HA6562_VEL']+galvel

amp_ratio = amp_ratio_from_flux_ratio(OII3729,OII3726,1.4)
velocity_gvar = gvar.gvar(invel, 20)
broadening_gvar = gvar.gvar(1, 20)

fitshape = 'sinc'

fit = fit_lines_in_spectrum(inspec, [OII3726, OII3729], 
                      step=cube.params.step, 
                      order=cube.params.order, 
                      nm_laser=cube.params.nm_laser, 
                      theta=corr2theta(cube.params.axis_corr), 
                      zpd_index=cube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def=['1', '1'],
                      pos_cov=[velocity_gvar],
                      amp_def=['1', '1'], 
                      amp_guess=[1, amp_ratio],
                      #sigma_def=['1', '1'],
                      #sigma_guess=[broadening_gvar,broadening_gvar],
                      #sigma_cov=10
                      #snr_guess=3
                      )

### Assign the fits to variables

In [ ]:
OII3726_fit, OII3729_fit = fit['fitted_models']['Cm1LinesModel']
OII3726red, OII3729red = fit['lines_params'][0][2], fit['lines_params'][1][2]

### Plot the fit results

In [ ]:
low, up = np.where(cube.params.filter_range[0] < waves)[0][0], np.where(cube.params.filter_range[1] < waves)[0][0]
red3729 = 1/((3729*(galvel)/(299792) + 3729) * 10**-8)

fig = plt.figure()
fig.set_size_inches(10, 4)

region = 0

plt.plot(waves, (specextr[brightreg[region]] * 10**-20) - specs, color='green', label=f'Sky Subtracted Region {brightreg[region]}')
plt.axhline(0, color='k', label='0 line')

plt.plot(waves, OII3726_fit, color='red', label='[OII]3726 fit')
plt.plot(waves, OII3729_fit, color='purple', label='[OII]3729 fit')
plt.plot(waves, OII3726_fit+OII3729_fit, c='k', label=f'[OII]3729 Fit + [OII3726] Fit')

plt.axvline(red3729, color='blue', linestyle='--', label='Redshifted 3729 Angstrom')
plt.xlim(np.min(waves[low:up]), np.max(waves[low:up]))
plt.xlim(OII3726red-150, OII3726red+150)
#plt.ylim(0, None)

plt.legend()
plt.title(f'{galaxy} sky background')
plt.xlabel(r'Wavenumber [cm$^{-1}$]')
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]')